# * VINSIGHT : Churn Subs
    Prepaid Churn Subs : TMH ยังใช้ไม่ได้ รอ verify ตัวเลข

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20240101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B1S000200' --Prepaid Churn Subs
        , 'TB1S000200' --Prepaid Churn Subs : TMH
        , 'DB1S000200' --Prepaid Churn Subs : DTAC
        
        , 'B2S010200' --Postpaid Churn Subs B2C
        , 'TB2S010200' --Postpaid Churn Subs B2C : TMH
        , 'DB2S010200' --Postpaid Churn Subs B2C : DTAC
        , 'B2S010201' --Postpaid Churn Subs Voluntary B2C
        , 'TB2S010201' --Postpaid Churn Subs Voluntary B2C : TMH
        , 'DB2S010201' --Postpaid Churn Subs Voluntary B2C : DTAC
        , 'B2S010202' --Postpaid Churn Subs Involuntary B2C
        , 'TB2S010202' --Postpaid Churn Subs Involuntary B2C : TMH
        , 'DB2S010202' --Postpaid Churn Subs Involuntary B2C : DTAC
        , 'TB2S010210' --Postpaid Churn Subs B2C : TMH (Most Used Location)
        , 'TB2S010211' --Postpaid Churn Subs Voluntary B2C : TMH (Most Used Location)
        , 'TB2S010212' --Postpaid Churn Subs Involuntary B2C : TMH (Most Used Location)
        
        , 'B2S020200' --Postpaid Churn Subs B2B
        , 'TB2S020200' --Postpaid Churn Subs B2B : TMH
        , 'DB2S020200' --Postpaid Churn Subs B2B : DTAC
        , 'B2S020201' --Postpaid Churn Subs Voluntary B2B
        , 'TB2S020201' --Postpaid Churn Subs Voluntary B2B : TMH
        , 'DB2S020201' --Postpaid Churn Subs Voluntary B2B : DTAC
        , 'B2S020202' --Postpaid Churn Subs Involuntary B2B
        , 'TB2S020202' --Postpaid Churn Subs Involuntary B2B : TMH
        , 'DB2S020202' --Postpaid Churn Subs Involuntary B2B : DTAC
        , 'TB2S020210' --Postpaid Churn Subs B2B : TMH (Most Used Location)
        , 'TB2S020211' --Postpaid Churn Subs Voluntary B2B : TMH (Most Used Location)
        , 'TB2S020212' --Postpaid Churn Subs Involuntary B2B : TMH (Most Used Location)
        
        , 'TSER13100' --FTTx Churn Subs
        , 'TSER13102' --FTTx Churn Subs Voluntary
        , 'TSER13103' --FTTx Churn Subs Involuntary
        , 'TSER13104' --FTTx Churn Subs Involuntary (Collection Forecast)
        , 'TSER14100' --TVS Churn Subs
        , 'TSER14102' --TVS Churn Subs Voluntary
        , 'TSER14103' --TVS Churn Subs Involuntary
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20240101

Data as of 2025-07-14, 10:59:56

DataFrame: 17360 rows, 16 columns


In [3]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift().max().astype(int)

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

# curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
# prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

## Overview by Period

In [4]:
''' Churn Subs Yearly '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'TB1S000200' #Prepaid Churn Subs : TMH
    , 'DB1S000200' #Prepaid Churn Subs : DTAC
    , 'B2S010200' #Postpaid Churn Subs B2C
    , 'TB2S010200' #Postpaid Churn Subs B2C : TMH
    , 'DB2S010200' #Postpaid Churn Subs B2C : DTAC
    , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    , 'TB2S010201' #Postpaid Churn Subs Voluntary B2C : TMH
    , 'DB2S010201' #Postpaid Churn Subs Voluntary B2C : DTAC
    , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TB2S010202' #Postpaid Churn Subs Involuntary B2C : TMH
    , 'DB2S010202' #Postpaid Churn Subs Involuntary B2C : DTAC
    # , 'TB2S010210' #Postpaid Churn Subs B2C : TMH (Most Used Location)
    # , 'TB2S010211' #Postpaid Churn Subs Voluntary B2C : TMH (Most Used Location)
    # , 'TB2S010212' #Postpaid Churn Subs Involuntary B2C : TMH (Most Used Location)
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    , 'TSER14102' #TVS Churn Subs Voluntary
    , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_sub_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
churn_sub_yearly_df = churn_sub_yearly_df.loc[churn_sub_yearly_df['METRIC_CD'].isin(v_metric_list)]

churn_sub_yearly_df['PRE'] = np.where(churn_sub_yearly_df['METRIC_CD']=='B1S000200', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['PRE_T'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TB1S000200', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['PRE_D'] = np.where(churn_sub_yearly_df['METRIC_CD']=='DB1S000200', churn_sub_yearly_df['P'], 0)

churn_sub_yearly_df['POST_B2C'] = np.where(churn_sub_yearly_df['METRIC_CD']=='B2S010200', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['POST_B2C_VL'] = np.where(churn_sub_yearly_df['METRIC_CD']=='B2S010201', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['POST_B2C_IVL'] = np.where(churn_sub_yearly_df['METRIC_CD']=='B2S010202', churn_sub_yearly_df['P'], 0)

churn_sub_yearly_df['POST_B2C_T'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TB2S010200', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['POST_B2C_VL_T'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TB2S010201', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['POST_B2C_IVL_T'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TB2S010202', churn_sub_yearly_df['P'], 0)

churn_sub_yearly_df['POST_B2C_D'] = np.where(churn_sub_yearly_df['METRIC_CD']=='DB2S010200', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['POST_B2C_VL_D'] = np.where(churn_sub_yearly_df['METRIC_CD']=='DB2S010201', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['POST_B2C_IVL_D'] = np.where(churn_sub_yearly_df['METRIC_CD']=='DB2S010202', churn_sub_yearly_df['P'], 0)

churn_sub_yearly_df['FTTX'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TSER13100', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['FTTX_VL'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TSER13102', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['FTTX_IVL'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TSER13103', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['FTTX_IVL(CF)'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TSER13104', churn_sub_yearly_df['P'], 0)

churn_sub_yearly_df['TVS'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TSER14100', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['TVS_VL'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TSER14102', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['TVS_IVL'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TSER14103', churn_sub_yearly_df['P'], 0)


# ''' Full '''
# churn_sub_yearly_df = churn_sub_yearly_df.groupby('TM_KEY_YR')\
#     .agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum'
#           , 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum'
#           , 'POST_B2C_T':'sum', 'POST_B2C_VL_T':'sum', 'POST_B2C_IVL_T':'sum'
#           , 'POST_B2C_D':'sum', 'POST_B2C_VL_D':'sum', 'POST_B2C_IVL_D':'sum'
#           , 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum'
#           , 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
# churn_sub_yearly_df = churn_sub_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
# churn_sub_yearly_df = churn_sub_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF'
#                                          , 'PRE', 'PRE_T', 'PRE_D'
#                                          , 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL'
#                                          , 'POST_B2C_T', 'POST_B2C_VL_T', 'POST_B2C_IVL_T'
#                                          , 'POST_B2C_D', 'POST_B2C_VL_D', 'POST_B2C_IVL_D'
#                                          , 'FTTX', 'FTTX_VL', 'FTTX_IVL'
#                                          , 'TVS', 'TVS_VL', 'TVS_IVL']]
# churn_sub_yearly_df_display = churn_sub_yearly_df.copy()
# churn_sub_yearly_df_display['ACTUAL_AS_OF'] = churn_sub_yearly_df_display['ACTUAL_AS_OF'].astype(int)
# mod_col_list = churn_sub_yearly_df_display.iloc[:, 3:].columns.tolist()
# for col in mod_col_list:
#     churn_sub_yearly_df_display[col] = churn_sub_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
# print(f'\n{churn_sub_yearly_df_display.to_string(max_cols=100)}') #max_rows=1000


''' Overview '''
churn_sub_yearly_df = churn_sub_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum', 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum', 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
churn_sub_yearly_df = churn_sub_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
churn_sub_yearly_df = churn_sub_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL', 'FTTX', 'FTTX_VL', 'FTTX_IVL', 'TVS', 'TVS_VL', 'TVS_IVL']]
churn_sub_yearly_df_display = churn_sub_yearly_df.copy()
churn_sub_yearly_df_display['ACTUAL_AS_OF'] = churn_sub_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = churn_sub_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_sub_yearly_df_display[col] = churn_sub_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
churn_sub_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,PRE,POST_B2C,POST_B2C_VL,POST_B2C_IVL,FTTX,FTTX_VL,FTTX_IVL,TVS,TVS_VL,TVS_IVL
0,2024,2025-07-14 05:03:33,20241231,0,"2,857,216","1,066,524","1,862,336","456,668","170,154","281,222","194,250","84,176","18,800"
1,2025,2025-07-14 05:03:33,20250712,0,"1,218,517","501,903","724,024","211,464","87,551","139,095","125,038","12,995",125


In [5]:
''' Churn Subs Quarterly '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'TB1S000200' #Prepaid Churn Subs : TMH
    , 'DB1S000200' #Prepaid Churn Subs : DTAC
    , 'B2S010200' #Postpaid Churn Subs B2C
    , 'TB2S010200' #Postpaid Churn Subs B2C : TMH
    , 'DB2S010200' #Postpaid Churn Subs B2C : DTAC
    , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    , 'TB2S010201' #Postpaid Churn Subs Voluntary B2C : TMH
    , 'DB2S010201' #Postpaid Churn Subs Voluntary B2C : DTAC
    , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TB2S010202' #Postpaid Churn Subs Involuntary B2C : TMH
    , 'DB2S010202' #Postpaid Churn Subs Involuntary B2C : DTAC
    # , 'TB2S010210' #Postpaid Churn Subs B2C : TMH (Most Used Location)
    # , 'TB2S010211' #Postpaid Churn Subs Voluntary B2C : TMH (Most Used Location)
    # , 'TB2S010212' #Postpaid Churn Subs Involuntary B2C : TMH (Most Used Location)
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    , 'TSER14102' #TVS Churn Subs Voluntary
    , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_sub_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
churn_sub_quarterly_df = churn_sub_quarterly_df.loc[churn_sub_quarterly_df['METRIC_CD'].isin(v_metric_list)]

churn_sub_quarterly_df['PRE'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='B1S000200', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['PRE_T'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TB1S000200', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['PRE_D'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='DB1S000200', churn_sub_quarterly_df['P'], 0)

churn_sub_quarterly_df['POST_B2C'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='B2S010200', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['POST_B2C_VL'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='B2S010201', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['POST_B2C_IVL'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='B2S010202', churn_sub_quarterly_df['P'], 0)

churn_sub_quarterly_df['POST_B2C_T'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TB2S010200', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['POST_B2C_VL_T'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TB2S010201', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['POST_B2C_IVL_T'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TB2S010202', churn_sub_quarterly_df['P'], 0)

churn_sub_quarterly_df['POST_B2C_D'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='DB2S010200', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['POST_B2C_VL_D'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='DB2S010201', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['POST_B2C_IVL_D'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='DB2S010202', churn_sub_quarterly_df['P'], 0)

churn_sub_quarterly_df['FTTX'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TSER13100', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['FTTX_VL'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TSER13102', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['FTTX_IVL'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TSER13103', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['FTTX_IVL(CF)'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TSER13104', churn_sub_quarterly_df['P'], 0)

churn_sub_quarterly_df['TVS'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TSER14100', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['TVS_VL'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TSER14102', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['TVS_IVL'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TSER14103', churn_sub_quarterly_df['P'], 0)


# ''' Full '''
# churn_sub_quarterly_df = churn_sub_quarterly_df.groupby('TM_KEY_QTR')\
#     .agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum'
#           , 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum'
#           , 'POST_B2C_T':'sum', 'POST_B2C_VL_T':'sum', 'POST_B2C_IVL_T':'sum'
#           , 'POST_B2C_D':'sum', 'POST_B2C_VL_D':'sum', 'POST_B2C_IVL_D':'sum'
#           , 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum'
#           , 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
# churn_sub_quarterly_df = churn_sub_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
# churn_sub_quarterly_df = churn_sub_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF'
#                                          , 'PRE', 'PRE_T', 'PRE_D'
#                                          , 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL'
#                                          , 'POST_B2C_T', 'POST_B2C_VL_T', 'POST_B2C_IVL_T'
#                                          , 'POST_B2C_D', 'POST_B2C_VL_D', 'POST_B2C_IVL_D'
#                                          , 'FTTX', 'FTTX_VL', 'FTTX_IVL'
#                                          , 'TVS', 'TVS_VL', 'TVS_IVL']]
# churn_sub_quarterly_df_display = churn_sub_quarterly_df.copy()
# churn_sub_quarterly_df_display['ACTUAL_AS_OF'] = churn_sub_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
# mod_col_list = churn_sub_quarterly_df_display.iloc[:, 3:].columns.tolist()
# for col in mod_col_list:
#     churn_sub_quarterly_df_display[col] = churn_sub_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
# print(f'\n{churn_sub_quarterly_df_display.to_string(max_cols=100)}') #max_rows=1000


''' Overview '''
churn_sub_quarterly_df = churn_sub_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum', 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum', 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
churn_sub_quarterly_df = churn_sub_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
churn_sub_quarterly_df = churn_sub_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL', 'FTTX', 'FTTX_VL', 'FTTX_IVL', 'TVS', 'TVS_VL', 'TVS_IVL']]
churn_sub_quarterly_df_display = churn_sub_quarterly_df.copy()
churn_sub_quarterly_df_display['ACTUAL_AS_OF'] = churn_sub_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = churn_sub_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_sub_quarterly_df_display[col] = churn_sub_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
churn_sub_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,PRE,POST_B2C,POST_B2C_VL,POST_B2C_IVL,FTTX,FTTX_VL,FTTX_IVL,TVS,TVS_VL,TVS_IVL
0,20241,2025-07-14 05:03:33,20240331,0,"871,395","318,353","576,878","118,742","43,223","72,945","48,913","37,318","13,347"
1,20242,2025-07-14 05:03:33,20240630,0,"718,051","262,796","454,251","114,261","43,677","70,807","40,970","35,580","5,046"
2,20243,2025-07-14 05:03:33,20240930,0,"664,232","248,339","433,905","118,872","44,269","67,909","48,795","5,040",355
3,20244,2025-07-14 05:03:33,20241231,0,"603,538","237,036","397,302","104,793","38,985","69,561","55,572","6,238",52
4,20251,2025-07-14 05:03:33,20250331,0,"553,494","225,501","341,127","104,815","38,083","68,600","47,799","5,854",35
5,20252,2025-07-14 05:03:33,20250630,0,"586,573","243,446","339,713","92,588","43,483","70,494","66,735","5,905",44
6,20253,2025-07-14 05:03:33,20250712,0,"78,450","32,956","43,184","14,061","5,985",1,"10,504","1,236",46


In [6]:
''' Churn Subs Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'TB1S000200' #Prepaid Churn Subs : TMH
    , 'DB1S000200' #Prepaid Churn Subs : DTAC
    , 'B2S010200' #Postpaid Churn Subs B2C
    , 'TB2S010200' #Postpaid Churn Subs B2C : TMH
    , 'DB2S010200' #Postpaid Churn Subs B2C : DTAC
    , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    , 'TB2S010201' #Postpaid Churn Subs Voluntary B2C : TMH
    , 'DB2S010201' #Postpaid Churn Subs Voluntary B2C : DTAC
    , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TB2S010202' #Postpaid Churn Subs Involuntary B2C : TMH
    , 'DB2S010202' #Postpaid Churn Subs Involuntary B2C : DTAC
    # , 'TB2S010210' #Postpaid Churn Subs B2C : TMH (Most Used Location)
    # , 'TB2S010211' #Postpaid Churn Subs Voluntary B2C : TMH (Most Used Location)
    # , 'TB2S010212' #Postpaid Churn Subs Involuntary B2C : TMH (Most Used Location)
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    , 'TSER14102' #TVS Churn Subs Voluntary
    , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_sub_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr].copy()
churn_sub_monthly_df = churn_sub_monthly_df.loc[churn_sub_monthly_df['METRIC_CD'].isin(v_metric_list)]

churn_sub_monthly_df['PRE'] = np.where(churn_sub_monthly_df['METRIC_CD']=='B1S000200', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['PRE_T'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TB1S000200', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['PRE_D'] = np.where(churn_sub_monthly_df['METRIC_CD']=='DB1S000200', churn_sub_monthly_df['P'], 0)

churn_sub_monthly_df['POST_B2C'] = np.where(churn_sub_monthly_df['METRIC_CD']=='B2S010200', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['POST_B2C_VL'] = np.where(churn_sub_monthly_df['METRIC_CD']=='B2S010201', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['POST_B2C_IVL'] = np.where(churn_sub_monthly_df['METRIC_CD']=='B2S010202', churn_sub_monthly_df['P'], 0)

churn_sub_monthly_df['POST_B2C_T'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TB2S010200', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['POST_B2C_VL_T'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TB2S010201', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['POST_B2C_IVL_T'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TB2S010202', churn_sub_monthly_df['P'], 0)

churn_sub_monthly_df['POST_B2C_D'] = np.where(churn_sub_monthly_df['METRIC_CD']=='DB2S010200', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['POST_B2C_VL_D'] = np.where(churn_sub_monthly_df['METRIC_CD']=='DB2S010201', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['POST_B2C_IVL_D'] = np.where(churn_sub_monthly_df['METRIC_CD']=='DB2S010202', churn_sub_monthly_df['P'], 0)

churn_sub_monthly_df['FTTX'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TSER13100', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['FTTX_VL'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TSER13102', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['FTTX_IVL'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TSER13103', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['FTTX_IVL(CF)'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TSER13104', churn_sub_monthly_df['P'], 0)

churn_sub_monthly_df['TVS'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TSER14100', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['TVS_VL'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TSER14102', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['TVS_IVL'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TSER14103', churn_sub_monthly_df['P'], 0)


# ''' Full '''
# churn_sub_monthly_df = churn_sub_monthly_df.groupby('TM_KEY_MTH')\
#     .agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum'
#           , 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum'
#           , 'POST_B2C_T':'sum', 'POST_B2C_VL_T':'sum', 'POST_B2C_IVL_T':'sum'
#           , 'POST_B2C_D':'sum', 'POST_B2C_VL_D':'sum', 'POST_B2C_IVL_D':'sum'
#           , 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum'
#           , 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
# churn_sub_monthly_df = churn_sub_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
# churn_sub_monthly_df = churn_sub_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF'
#                                          , 'PRE', 'PRE_T', 'PRE_D'
#                                          , 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL'
#                                          , 'POST_B2C_T', 'POST_B2C_VL_T', 'POST_B2C_IVL_T'
#                                          , 'POST_B2C_D', 'POST_B2C_VL_D', 'POST_B2C_IVL_D'
#                                          , 'FTTX', 'FTTX_VL', 'FTTX_IVL'
#                                          , 'TVS', 'TVS_VL', 'TVS_IVL']]
# churn_sub_monthly_df_display = churn_sub_monthly_df.copy()
# churn_sub_monthly_df_display['ACTUAL_AS_OF'] = churn_sub_monthly_df_display['ACTUAL_AS_OF'].astype(int)
# mod_col_list = churn_sub_monthly_df_display.iloc[:, 3:].columns.tolist()
# for col in mod_col_list:
#     churn_sub_monthly_df_display[col] = churn_sub_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
# print(f'\n{churn_sub_monthly_df_display.to_string(max_cols=100)}') #max_rows=1000


''' Overview '''
churn_sub_monthly_df = churn_sub_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum', 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum', 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
churn_sub_monthly_df = churn_sub_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
churn_sub_monthly_df = churn_sub_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL', 'FTTX', 'FTTX_VL', 'FTTX_IVL', 'TVS', 'TVS_VL', 'TVS_IVL']]
churn_sub_monthly_df_display = churn_sub_monthly_df.copy()
churn_sub_monthly_df_display['ACTUAL_AS_OF'] = churn_sub_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = churn_sub_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_sub_monthly_df_display[col] = churn_sub_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
churn_sub_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,PRE,POST_B2C,POST_B2C_VL,POST_B2C_IVL,FTTX,FTTX_VL,FTTX_IVL,TVS,TVS_VL,TVS_IVL
0,202401,2025-07-14 05:03:33,20240131,0,"335,195","118,348","231,754","40,468","16,454","24,014","14,004","10,196","3,820"
1,202402,2025-07-14 05:03:33,20240229,0,"281,024","102,361","180,543","38,531","14,057","24,474","12,774","7,689","6,795"
2,202403,2025-07-14 05:03:33,20240331,0,"255,176","97,644","164,581","39,743","12,712","24,457","22,135","19,433","2,732"
3,202404,2025-07-14 05:03:33,20240430,0,"247,654","90,553","155,090","38,455","13,522","24,520","20,819","18,602","2,217"
4,202405,2025-07-14 05:03:33,20240531,0,"250,447","92,059","151,641","38,397","14,955","23,442","16,541","14,731","1,810"
5,202406,2025-07-14 05:03:33,20240630,0,"219,950","80,184","147,520","37,409","15,200","22,845","3,610","2,247","1,019"
6,202407,2025-07-14 05:03:33,20240731,0,"232,711","91,260","145,540","40,306","15,934","22,944","9,003","1,136",22
7,202408,2025-07-14 05:03:33,20240831,0,"224,948","81,816","143,964","40,742","14,958","22,914","21,101","2,225",313
8,202409,2025-07-14 05:03:33,20240930,0,"206,573","75,263","144,401","37,824","13,377","22,051","18,691","1,679",20
9,202410,2025-07-14 05:03:33,20241031,0,"207,481","83,683","135,820","38,097","13,580","23,214","19,574","2,181",30


In [7]:
''' Churn Subs Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'TB1S000200' #Prepaid Churn Subs : TMH
    , 'DB1S000200' #Prepaid Churn Subs : DTAC
    , 'B2S010200' #Postpaid Churn Subs B2C
    , 'TB2S010200' #Postpaid Churn Subs B2C : TMH
    , 'DB2S010200' #Postpaid Churn Subs B2C : DTAC
    , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    , 'TB2S010201' #Postpaid Churn Subs Voluntary B2C : TMH
    , 'DB2S010201' #Postpaid Churn Subs Voluntary B2C : DTAC
    , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TB2S010202' #Postpaid Churn Subs Involuntary B2C : TMH
    , 'DB2S010202' #Postpaid Churn Subs Involuntary B2C : DTAC
    # , 'TB2S010210' #Postpaid Churn Subs B2C : TMH (Most Used Location)
    # , 'TB2S010211' #Postpaid Churn Subs Voluntary B2C : TMH (Most Used Location)
    # , 'TB2S010212' #Postpaid Churn Subs Involuntary B2C : TMH (Most Used Location)
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    , 'TSER14102' #TVS Churn Subs Voluntary
    , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_sub_weekly_df = chk_src_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
churn_sub_weekly_df = churn_sub_weekly_df.loc[churn_sub_weekly_df['METRIC_CD'].isin(v_metric_list)]

churn_sub_weekly_df['PRE'] = np.where(churn_sub_weekly_df['METRIC_CD']=='B1S000200', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['PRE_T'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TB1S000200', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['PRE_D'] = np.where(churn_sub_weekly_df['METRIC_CD']=='DB1S000200', churn_sub_weekly_df['P'], 0)

churn_sub_weekly_df['POST_B2C'] = np.where(churn_sub_weekly_df['METRIC_CD']=='B2S010200', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['POST_B2C_VL'] = np.where(churn_sub_weekly_df['METRIC_CD']=='B2S010201', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['POST_B2C_IVL'] = np.where(churn_sub_weekly_df['METRIC_CD']=='B2S010202', churn_sub_weekly_df['P'], 0)

churn_sub_weekly_df['POST_B2C_T'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TB2S010200', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['POST_B2C_VL_T'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TB2S010201', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['POST_B2C_IVL_T'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TB2S010202', churn_sub_weekly_df['P'], 0)

churn_sub_weekly_df['POST_B2C_D'] = np.where(churn_sub_weekly_df['METRIC_CD']=='DB2S010200', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['POST_B2C_VL_D'] = np.where(churn_sub_weekly_df['METRIC_CD']=='DB2S010201', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['POST_B2C_IVL_D'] = np.where(churn_sub_weekly_df['METRIC_CD']=='DB2S010202', churn_sub_weekly_df['P'], 0)

churn_sub_weekly_df['FTTX'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TSER13100', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['FTTX_VL'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TSER13102', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['FTTX_IVL'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TSER13103', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['FTTX_IVL(CF)'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TSER13104', churn_sub_weekly_df['P'], 0)

churn_sub_weekly_df['TVS'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TSER14100', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['TVS_VL'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TSER14102', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['TVS_IVL'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TSER14103', churn_sub_weekly_df['P'], 0)


# ''' Full '''
# churn_sub_weekly_df = churn_sub_weekly_df.groupby('TM_KEY_WK')\
#     .agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum'
#           , 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum'
#           , 'POST_B2C_T':'sum', 'POST_B2C_VL_T':'sum', 'POST_B2C_IVL_T':'sum'
#           , 'POST_B2C_D':'sum', 'POST_B2C_VL_D':'sum', 'POST_B2C_IVL_D':'sum'
#           , 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum'
#           , 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
# churn_sub_weekly_df = churn_sub_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
# churn_sub_weekly_df = churn_sub_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF'
#                                          , 'PRE', 'PRE_T', 'PRE_D'
#                                          , 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL'
#                                          , 'POST_B2C_T', 'POST_B2C_VL_T', 'POST_B2C_IVL_T'
#                                          , 'POST_B2C_D', 'POST_B2C_VL_D', 'POST_B2C_IVL_D'
#                                          , 'FTTX', 'FTTX_VL', 'FTTX_IVL'
#                                          , 'TVS', 'TVS_VL', 'TVS_IVL']]
# churn_sub_weekly_df_display = churn_sub_weekly_df.copy()
# churn_sub_weekly_df_display['ACTUAL_AS_OF'] = churn_sub_weekly_df_display['ACTUAL_AS_OF'].astype(int)
# mod_col_list = churn_sub_weekly_df_display.iloc[:, 3:].columns.tolist()
# for col in mod_col_list:
#     churn_sub_weekly_df_display[col] = churn_sub_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
# print(f'\n{churn_sub_weekly_df_display.to_string(max_cols=100)}') #max_rows=1000


''' Overview '''
churn_sub_weekly_df = churn_sub_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum', 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum', 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
churn_sub_weekly_df = churn_sub_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
churn_sub_weekly_df = churn_sub_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL', 'FTTX', 'FTTX_VL', 'FTTX_IVL', 'TVS', 'TVS_VL', 'TVS_IVL']]
churn_sub_weekly_df_display = churn_sub_weekly_df.copy()
churn_sub_weekly_df_display['ACTUAL_AS_OF'] = churn_sub_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = churn_sub_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_sub_weekly_df_display[col] = churn_sub_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
churn_sub_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,PRE,POST_B2C,POST_B2C_VL,POST_B2C_IVL,FTTX,FTTX_VL,FTTX_IVL,TVS,TVS_VL,TVS_IVL
0,2025022,2025-07-14 05:03:33,20250601,0,"4,299","2,960","2,275",-814,641,0,"1,142",146,0
1,2025023,2025-07-14 05:03:33,20250608,0,"49,654","23,579","27,180","14,695","3,956","1,905","6,653",630,1
2,2025024,2025-07-14 05:03:33,20250615,0,"48,846","19,958","24,527","12,507","3,525","8,490","5,858",402,0
3,2025025,2025-07-14 05:03:33,20250622,0,"41,706","17,886","25,819","8,670","3,444","4,097","5,560",372,1
4,2025026,2025-07-14 05:03:33,20250629,0,"51,597","18,454","31,336","10,939","3,552","9,312","6,646",574,2
5,2025027,2025-07-14 05:03:33,20250706,0,"43,938","21,848","28,836","12,049","4,459",8,"7,381","1,056",1
6,2025028,2025-07-14 05:03:33,20250712,0,"39,102","13,978","17,767",0,"2,204",0,"4,343",334,45


In [8]:
''' Churn Subs Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'TB1S000200' #Prepaid Churn Subs : TMH
    , 'DB1S000200' #Prepaid Churn Subs : DTAC
    , 'B2S010200' #Postpaid Churn Subs B2C
    , 'TB2S010200' #Postpaid Churn Subs B2C : TMH
    , 'DB2S010200' #Postpaid Churn Subs B2C : DTAC
    , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    , 'TB2S010201' #Postpaid Churn Subs Voluntary B2C : TMH
    , 'DB2S010201' #Postpaid Churn Subs Voluntary B2C : DTAC
    , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TB2S010202' #Postpaid Churn Subs Involuntary B2C : TMH
    , 'DB2S010202' #Postpaid Churn Subs Involuntary B2C : DTAC
    # , 'TB2S010210' #Postpaid Churn Subs B2C : TMH (Most Used Location)
    # , 'TB2S010211' #Postpaid Churn Subs Voluntary B2C : TMH (Most Used Location)
    # , 'TB2S010212' #Postpaid Churn Subs Involuntary B2C : TMH (Most Used Location)
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    , 'TSER14102' #TVS Churn Subs Voluntary
    , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_sub_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
churn_sub_daily_df = churn_sub_daily_df.loc[churn_sub_daily_df['METRIC_CD'].isin(v_metric_list)]

churn_sub_daily_df['PRE'] = np.where(churn_sub_daily_df['METRIC_CD']=='B1S000200', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['PRE_T'] = np.where(churn_sub_daily_df['METRIC_CD']=='TB1S000200', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['PRE_D'] = np.where(churn_sub_daily_df['METRIC_CD']=='DB1S000200', churn_sub_daily_df['P'], 0)

churn_sub_daily_df['POST_B2C'] = np.where(churn_sub_daily_df['METRIC_CD']=='B2S010200', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_VL'] = np.where(churn_sub_daily_df['METRIC_CD']=='B2S010201', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_IVL'] = np.where(churn_sub_daily_df['METRIC_CD']=='B2S010202', churn_sub_daily_df['P'], 0)

churn_sub_daily_df['POST_B2C_T'] = np.where(churn_sub_daily_df['METRIC_CD']=='TB2S010200', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_VL_T'] = np.where(churn_sub_daily_df['METRIC_CD']=='TB2S010201', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_IVL_T'] = np.where(churn_sub_daily_df['METRIC_CD']=='TB2S010202', churn_sub_daily_df['P'], 0)

churn_sub_daily_df['POST_B2C_D'] = np.where(churn_sub_daily_df['METRIC_CD']=='DB2S010200', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_VL_D'] = np.where(churn_sub_daily_df['METRIC_CD']=='DB2S010201', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_IVL_D'] = np.where(churn_sub_daily_df['METRIC_CD']=='DB2S010202', churn_sub_daily_df['P'], 0)

churn_sub_daily_df['FTTX'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER13100', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['FTTX_VL'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER13102', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['FTTX_IVL'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER13103', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['FTTX_IVL(CF)'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER13104', churn_sub_daily_df['P'], 0)

churn_sub_daily_df['TVS'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER14100', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['TVS_VL'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER14102', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['TVS_IVL'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER14103', churn_sub_daily_df['P'], 0)


# ''' Full '''
# churn_sub_daily_df = churn_sub_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY'])\
#     .agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum'
#           , 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum'
#           , 'POST_B2C_T':'sum', 'POST_B2C_VL_T':'sum', 'POST_B2C_IVL_T':'sum'
#           , 'POST_B2C_D':'sum', 'POST_B2C_VL_D':'sum', 'POST_B2C_IVL_D':'sum'
#           , 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum'
#           , 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
# churn_sub_daily_df = churn_sub_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
# churn_sub_daily_df = churn_sub_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM'
#                                          , 'PRE', 'PRE_T', 'PRE_D'
#                                          , 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL'
#                                          , 'POST_B2C_T', 'POST_B2C_VL_T', 'POST_B2C_IVL_T'
#                                          , 'POST_B2C_D', 'POST_B2C_VL_D', 'POST_B2C_IVL_D'
#                                          , 'FTTX', 'FTTX_VL', 'FTTX_IVL'
#                                          , 'TVS', 'TVS_VL', 'TVS_IVL']]
# churn_sub_daily_df_display = churn_sub_daily_df.copy()
# mod_col_list = churn_sub_daily_df_display.iloc[:, 3:].columns.tolist()
# for col in mod_col_list:
#     churn_sub_daily_df_display[col] = churn_sub_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
# print(f'\n{churn_sub_daily_df_display.to_string(max_cols=100)}') #max_rows=1000


''' Overview '''
churn_sub_daily_df = churn_sub_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum', 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum', 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
churn_sub_daily_df = churn_sub_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
churn_sub_daily_df = churn_sub_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL', 'FTTX', 'FTTX_VL', 'FTTX_IVL', 'TVS', 'TVS_VL', 'TVS_IVL']]
churn_sub_daily_df_display = churn_sub_daily_df.copy()
mod_col_list = churn_sub_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_sub_daily_df_display[col] = churn_sub_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
churn_sub_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE,POST_B2C,POST_B2C_VL,POST_B2C_IVL,FTTX,FTTX_VL,FTTX_IVL,TVS,TVS_VL,TVS_IVL
0,202507,20250713,2025-07-14 05:03:33,0,0,0,0,0,0,0,0,0,0
1,202507,20250712,2025-07-14 05:03:33,0,0,0,0,0,0,0,0,0,0
2,202507,20250711,2025-07-14 05:03:33,0,"10,229","1,641","2,400",0,0,0,822,50,0
3,202507,20250710,2025-07-14 05:03:33,0,"3,682","1,869","3,206",0,542,0,"1,016",30,28
4,202507,20250709,2025-07-14 05:03:33,0,"4,695","2,557","3,973",0,568,0,881,52,4
5,202507,20250708,2025-07-14 05:03:33,0,"8,269","5,301","4,384",0,525,0,879,74,13
6,202507,20250707,2025-07-14 05:03:33,0,"12,227","2,610","3,804",0,569,0,745,128,0
7,202507,20250706,2025-07-14 05:03:33,0,"2,217","2,321","3,293","5,686",620,0,"1,197",105,1
8,202507,20250705,2025-07-14 05:03:33,0,"6,534","2,083","5,300",-176,621,0,"1,095",158,0
9,202507,20250704,2025-07-14 05:03:33,0,"4,264","2,667","2,876",-385,608,0,893,71,0


## Products Summary

### Prep Monthly Data

In [9]:
''' Monthly Summary '''

monthly_df = chk_src_df.copy()
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display
# monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]

In [10]:
''' Parameter '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth
# v_tm_key_mth = 202505

print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202506


### Prepaid

In [11]:
''' Prepaid '''

v_product_grp = 'Prepaid'

prepaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_df = prepaid_df.reset_index(drop=True)
prepaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)


### Postpaid

In [12]:
''' Postpaid '''

v_product_grp = 'Postpaid'

postpaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_df = postpaid_df.loc[~postpaid_df['METRIC_NAME'].str.contains('B2C|B2B')]
postpaid_df = postpaid_df.loc[postpaid_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_df = postpaid_df.reset_index(drop=True)
postpaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)


In [13]:
''' Postpaid B2C '''

v_product_grp = 'Postpaid'

postpaid_b2c_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_df = postpaid_b2c_df.reset_index(drop=True)
postpaid_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202506,Postpaid,B2S010200,Postpaid Churn Subs B2C,2025-07-14 05:03:33,"200,692","200,692","200,419","200,419","200,421",0
1,202506,Postpaid,B2S010201,Postpaid Churn Subs Voluntary B2C,2025-07-14 05:03:33,"85,707","85,707","85,439","85,439","85,443",0
2,202506,Postpaid,B2S010202,Postpaid Churn Subs Involuntary B2C,2025-07-14 05:03:33,"114,556","114,556","114,553","114,553","114,553",0
3,202506,Postpaid,DB2S010200,Postpaid Churn Subs B2C : DTAC,2025-07-14 05:03:33,"59,845","59,845","59,783","59,783","59,783",0
4,202506,Postpaid,DB2S010201,Postpaid Churn Subs Voluntary B2C : DTAC,2025-07-14 05:03:33,"22,695","22,695","22,636","22,636","22,636",0
5,202506,Postpaid,DB2S010202,Postpaid Churn Subs Involuntary B2C : DTAC,2025-07-14 05:03:33,"36,721","36,721","36,720","36,720","36,720",0
6,202506,Postpaid,TB2S010200,Postpaid Churn Subs B2C : TMH,2025-07-14 05:03:33,"140,847","140,847","140,636","140,636","140,638",0
7,202506,Postpaid,TB2S010201,Postpaid Churn Subs Voluntary B2C : TMH,2025-07-14 05:03:33,"63,012","63,012","62,803","62,803","62,807",0
8,202506,Postpaid,TB2S010202,Postpaid Churn Subs Involuntary B2C : TMH,2025-07-14 05:03:33,"77,835","77,835","77,833","77,833","77,833",0
9,202506,Postpaid,TB2S010210,Postpaid Churn Subs B2C : TMH (Most Used Locat...,2025-07-14 05:03:33,0,0,0,0,0,0


In [14]:
''' Postpaid B2B '''

v_product_grp = 'Postpaid'

postpaid_b2b_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_df = postpaid_b2b_df.reset_index(drop=True)
postpaid_b2b_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202506,Postpaid,B2S020200,Postpaid Churn Subs B2B,2025-07-14 03:43:50,0,"52,686","52,685","52,682",0,"-52,686"
1,202506,Postpaid,B2S020201,Postpaid Churn Subs Voluntary B2B,2025-07-14 03:43:50,0,"51,635","51,634","51,631",0,"-51,635"
2,202506,Postpaid,B2S020202,Postpaid Churn Subs Involuntary B2B,2025-07-14 03:43:50,0,"1,051","1,051","1,051",0,"-1,051"
3,202506,Postpaid,DB2S020200,Postpaid Churn Subs B2B : DTAC,2025-07-14 03:43:50,0,"12,224","12,224","12,221",0,"-12,224"
4,202506,Postpaid,DB2S020201,Postpaid Churn Subs Voluntary B2B : DTAC,2025-07-14 03:43:50,0,"11,713","11,713","11,710",0,"-11,713"
5,202506,Postpaid,DB2S020202,Postpaid Churn Subs Involuntary B2B : DTAC,2025-07-14 03:43:50,0,511,511,511,0,-511
6,202506,Postpaid,TB2S020200,Postpaid Churn Subs B2B : TMH,2025-07-14 05:03:33,0,"40,462","40,461","40,461","40,461","-40,462"
7,202506,Postpaid,TB2S020201,Postpaid Churn Subs Voluntary B2B : TMH,2025-07-14 05:03:33,0,"39,922","39,921","39,921","39,921","-39,922"
8,202506,Postpaid,TB2S020202,Postpaid Churn Subs Involuntary B2B : TMH,2025-07-14 05:03:33,0,540,540,540,540,-540
9,202506,Postpaid,TB2S020210,Postpaid Churn Subs B2B : TMH (Most Used Locat...,2025-07-14 05:03:33,0,"40,462","40,461","40,461","40,461","-40,462"


### TOL

In [15]:
''' TOL '''

v_product_grp = 'TOL'

tol_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_df = tol_df.loc[tol_df['TM_KEY_MTH']==v_tm_key_mth]
tol_df = tol_df.reset_index(drop=True)
tol_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202506,TOL,TSER13100,FTTx Churn Subs,2025-07-14 05:03:33,0,"43,985","43,985","43,985","43,985","-43,985"
1,202506,TOL,TSER13102,FTTx Churn Subs Voluntary,2025-07-14 05:03:33,0,"15,796","15,796","15,796","15,796","-15,796"
2,202506,TOL,TSER13103,FTTx Churn Subs Involuntary,2025-07-14 05:03:33,0,"23,811","23,811","23,811","23,811","-23,811"
3,202506,TOL,TSER13104,FTTx Churn Subs Involuntary (Collection Forecast),2025-07-14 05:03:33,0,"24,435","24,435","24,435","24,435","-24,435"


### TVS

In [16]:
''' TVS '''

v_product_grp = 'TVS'

tvs_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_df = tvs_df.loc[tvs_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_df = tvs_df.reset_index(drop=True)
tvs_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202506,TVS,TSER14100,TVS Churn Subs,2025-07-14 03:43:50,0,"27,079","27,079","26,099",0,"-27,079"
1,202506,TVS,TSER14102,TVS Churn Subs Voluntary,2025-07-14 03:43:50,0,"2,278","27,075","26,099",0,"-2,278"
2,202506,TVS,TSER14103,TVS Churn Subs Involuntary,2025-07-14 03:43:50,0,4,4,1,0,-4


## ** Current Issue

In [17]:
''' Monthly : Prepaid Churn Subs : TMH (Align with BU) '''

v_metric_cd = 'TB1S000200'

issue_prepaid_tmh_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_prepaid_tmh_df = issue_prepaid_tmh_df.loc[issue_prepaid_tmh_df['TM_KEY_MTH']>=202401]
issue_prepaid_tmh_df = issue_prepaid_tmh_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)

issue_prepaid_tmh_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
